In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import keras

config = ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True

keras.backend.set_session(InteractiveSession(config=config))
session = InteractiveSession(config=config)

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator #Augmentation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train)
        std = np.std(X_train)
        X_train = (X_train-mean)/(std+1e-7)  #1e-7為1*10的-7次方,這是為了防止當std=0時會除以0
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()


classifier = Sequential()

#卷積組合
classifier.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())   # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, (3,3), activation='relu'))
classifier.add(BatchNormalization())  # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu',
                     kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])



img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
                             rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=False, dtype=np.float32)
img_gen.fit(x_train)



earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

# 開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                         epochs=100, validation_data = (x_test, y_test),
                         callbacks = [earlystop])

(50000, 32, 32, 3)










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100
500/500 [==============================] - 28s 56ms/step - loss: 2.1202 - acc: 0.3521 - val_loss: 1.6764 - val_acc: 0.4819
Epoch 2/100
 15/500 [..............................] - ETA: 6s - loss: 1.7742 - acc: 0.4387

/home/shinfu/anaconda3/envs/yolo_v1/lib/python3.6/site-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 19s 38ms/step - loss: 1.6609 - acc: 0.4846 - val_loss: 1.3770 - val_acc: 0.5829
Epoch 3/100
500/500 [==============================] - 18s 36ms/step - loss: 1.4842 - acc: 0.5384 - val_loss: 1.3298 - val_acc: 0.5823
Epoch 4/100
500/500 [==============================] - 18s 36ms/step - loss: 1.3774 - acc: 0.5688 - val_loss: 1.3137 - val_acc: 0.5920
Epoch 5/100
500/500 [==============================] - 20s 40ms/step - loss: 1.3055 - acc: 0.5889 - val_loss: 1.1216 - val_acc: 0.6500
Epoch 6/100
500/500 [==============================] - 18s 36ms/step - loss: 1.2586 - acc: 0.6035 - val_loss: 1.1745 - val_acc: 0.6330
Epoch 7/100
500/500 [==============================] - 18s 36ms/step - loss: 1.2156 - acc: 0.6172 - val_loss: 1.1101 - val_acc: 0.6515
Epoch 8/100
500/500 [==============================] - 18s 36ms/step - loss: 1.1973 - acc: 0.6248 - val_loss: 1.0162 - val_acc: 0.6819
Epoch 9/100
500/500 [==============================] - 18s 36ms/ste

KeyboardInterrupt: 